In [ ]:
!pip install sentence_transformers
!pip install datasets
!pip install pinecone-client
!pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from pinecone import Pinecone, ServerlessSpec
import os
from tqdm import tqdm
import cohere
import numpy as np
import warnings
from IPython.display import display
warnings.filterwarnings("ignore")

In [4]:
with open("chohere_api_keys.txt") as f:
    COHERE_API_KEY = f.read().strip()
with open("pinecone_api_key.txt") as f:
    PINECONE_API_KEY = f.read().strip()

In [5]:
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL = 'all-MiniLM-L6-v2'
model = SentenceTransformer(EMBEDDING_MODEL)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
def chunk_docs(
        documents: list,
        max_chunk_length: int = 300,
        stride: int = 100
    ) -> list:
    """
    Chunk documents to multiple chunks of length lower or equal to
    max_chunk_length and start in gaps equal to stride.
    Args:
        documents: List of documents as strings to chunk.
        max_chunk_length: Maximum length of a chunk.
        stride: Gap size to move the chunk window.
    Returns:
        List of chunked documents.
    """
    chunked_docs = []

    for document in documents:
        doc_len = len(document)
        if doc_len <= max_chunk_length:
            chunked_docs.append(document)
        else:
            start = 0
            while start < doc_len:
                end = min(start + max_chunk_length, doc_len)
                chunk = document[start:end]
                chunked_docs.append(chunk)
                start += stride

    return chunked_docs

In [14]:
def load_and_embedd_dataset(
        dataset_name: str = 'nihal-mp/worst_cars_in_history',
        split: str = 'train',
        model: SentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2'),
        text_field: str = 'Content',
        max_chunk_length: int = 300,
        stride: int = 100,
        rec_num: int = 400
) -> tuple:
    """
    Load a dataset and embedd the text field using a sentence-transformer model
    Args:
        dataset_name: The name of the dataset to load
        split: The split of the dataset to load
        model: The model to use for embedding
        text_field: The field in the dataset that contains the text
        rec_num: The number of records to load and embedd
    Returns:
        tuple: A tuple containing the chunked documents and the embeddings
    """
    from datasets import load_dataset

    print("Loading and embedding the dataset")

    # Load the dataset
    dataset = load_dataset(dataset_name, split=split)

    # Chunk the documents
    used_documents = dataset[text_field][:rec_num]
    chunked_docs = chunk_docs(documents=used_documents)

    # Embed the first `rec_num` rows of the dataset
    embeddings = model.encode(chunked_docs)

    print("Done!")
    return dataset, chunked_docs, embeddings

In [16]:
DATASET_NAME = 'nihal-mp/worst_cars_in_history'

dataset, chunked_docs, embeddings = load_and_embedd_dataset(
    dataset_name=DATASET_NAME,
    rec_num=40,
    model=model,
)
shape = embeddings.shape

Loading and embedding the dataset
Done!


Lets us look at the dataset and the embeddings

In [17]:
pd_dataset = dataset.to_pandas()
pd_dataset.head(5)

,Car,Content
0,Triumph Mayflower (1949-53),Triumph Mayflower The Triumph Mayflower was an...
1,Nash/Austin Metropolitan (1954-62),Nash Metropolitan The Nash Metropolitan was on...
2,Renault Dauphine (North American version) (195...,Renault Dauphine While the Renault Dauphine wa...
3,Trabant (1957-90),Trabant P50 Limousine The Trabant P50 was intr...
4,Edsel (1958),Edsel Corsair Launched with considerable publi...


In [18]:
print(f"The embeddings shape: {embeddings.shape}")

The embeddings shape: (845, 384)


## Second Element - Vector Database
We will use Pinecone's free-to-use vectorDB

In [19]:
def create_pinecone_index(
        index_name: str,
        dimension: int,
        metric: str = 'cosine',
):
    """
    Create a pinecone index if it does not exist
    Args:
        index_name: The name of the index
        dimension: The dimension of the index
        metric: The metric to use for the index
    Returns:
        Pinecone: A pinecone object which can later be used for upserting vectors and connecting to VectorDBs
    """
    from pinecone import Pinecone, ServerlessSpec
    print("Creating a Pinecone index...")
    pc = Pinecone(api_key=PINECONE_API_KEY)
    existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
    if index_name not in existing_indexes:
        pc.create_index(
            name=index_name,
            dimension=dimension,
            # Remember! It is crucial that the metric you will use in your VectorDB will also be a metric your embedding
            # model works well with!
            metric=metric,
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
    print("Done!")
    return pc

In [20]:
INDEX_NAME = 'worst-cars-in-history'

# Create the vector database
# We are passing the index_name and the size of our embeddings
pc = create_pinecone_index(INDEX_NAME, shape[1])

Creating a Pinecone index...
Done!


Now that we have created the vector database, let's add some data to it!

In [24]:
def upsert_vectors(
        index: Pinecone,
        embeddings: np.ndarray,
        chunked_docs: list,
        text_field: str = 'Content',
        batch_size: int = 128
):
    """
    Upsert vectors to a pinecone index
    Args:
        index: The pinecone index object
        embeddings: The embeddings to upsert
        dataset: The dataset containing the metadata
        batch_size: The batch size to use for upserting
    Returns:
        An updated pinecone index
    """
    print("Upserting the embeddings to the Pinecone index...")
    shape = embeddings.shape

    ids = [str(i) for i in range(shape[0])]
    meta = [{text_field: text} for text in chunked_docs]

    # create list of (id, vector, metadata) tuples to be upserted
    to_upsert = list(zip(ids, embeddings, meta))

    for i in tqdm(range(0, shape[0], batch_size)):
        i_end = min(i + batch_size, shape[0])
        index.upsert(vectors=to_upsert[i:i_end])
    return index


In [25]:
# Upsert the embeddings to the Pinecone index
index = pc.Index(INDEX_NAME)
index_upserted = upsert_vectors(index, embeddings, chunked_docs)

Upserting the embeddings to the Pinecone index...


100%|██████████| 7/7 [00:03<00:00,  1.76it/s]


Let's view the index statistics!

In [31]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 845}},
 'total_vector_count': 845}

## Third Element - LLM
We will use [Cohere's chat API](https://cohere.com/chat)

In [33]:
import cohere

#First lets write a query for the LLM
query = "In which stuff.co.nz list was the Nash metropolitan included?"

co = cohere.Client(api_key=COHERE_API_KEY)
response = co.chat(
        model='command-r-plus',
        message=query,
    )
response.text

'The Nash Metropolitan was included in a list of "The 10 weirdest and wackiest cars ever made" published by stuff.co.nz.'

In [34]:
def augment_prompt(
        query: str,
        model: SentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2'),
        index=None,
) -> str:
    """
    Augment the prompt with the top 3 results from the knowledge base
    Args:
        query: The query to augment
        index: The vectorstore object
    Returns:
        str: The augmented prompt
    """
    results = [float(val) for val in list(model.encode(query))]

    # get top 3 results from knowledge base
    query_results = index.query(
        vector=results,
        top_k=3,
        include_values=True,
        include_metadata=True
    )['matches']
    text_matches = [match['metadata']['Content'] for match in query_results]

    # get the text from the results
    source_knowledge = "\n\n".join(text_matches)

    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.
    Contexts:
    {source_knowledge}
    If the answer is not included in the source knowledge - say that you don't know.
    Query: {query}"""
    return augmented_prompt, source_knowledge

In [35]:
# Let us remember our query
query = "In which stuff.co.nz list was the Nash metropolitan included?"
augmented_prompt, source_knowledge = augment_prompt(query, model=model, index=index)
response = co.chat(
        model='command-r-plus',
        message=augmented_prompt,
    )
response.text

'The Nash Metropolitan was included in the Stuff.co.nz list of "Cars that Should Never Have Been Built."'

In [36]:
print(source_knowledge)

l of The Independent called it "one of the nastiest cars ever built."[17] Stuff.co.nz included in its list of "Cars that should never have been built," saying: "Designed to conquer the US market by combining Nash styling with British small car mechanicals and innovation, the Metropolitan was an abje

Nash Metropolitan The Nash Metropolitan was one of the first attempts by a US car maker to produce a small car "which would be mainly used as a second car."[13] Compared to other small cars of the era, it was to be much more luxurious, intended to be "a big car in miniature" by its main developer Ge

mbining Nash styling with British small car mechanicals and innovation, the Metropolitan was an abject failure being the worst of both worlds."[9] Money Inc. included it in its list of the 20 worst cars ever made, calling it "One of the biggest bombs in automotive history and definitely one of the w
